In [16]:
%load_ext dotenv
%dotenv -o

import os
os.environ["POLYGON_DATA_DIR"]

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


'/media/mirror/files.polygon.io'

In [17]:
from zipline_polygon_bundle import get_ticker_universe
from zipline_polygon_bundle.config import PolygonConfig

# from zipline_polygon_bundle import date_to_path, trades_schema, cast_trades, custom_aggs_schema, custom_aggs_partitioning, generate_csv_trades_tables, trades_to_custom_aggs
# from zipline_polygon_bundle import get_custom_aggs_dates

from typing import Iterator, Tuple

import pandas as pd
import plotly.graph_objs as go

import pyarrow as pa
from pyarrow import dataset as pa_ds
from pyarrow import compute as pa_compute
from pyarrow import compute as pc
from pyarrow import parquet as pa_parquet

from fsspec.implementations.arrow import ArrowFSWrapper

import os

import datetime
import pandas_market_calendars
import numpy as np

import resource

from itables import show

In [18]:
os.environ["CUSTOM_ASSET_FILES_DIR"]

'/home/jovyan/work/data'

In [19]:
if not "CUSTOM_ASSET_FILES_DIR" in os.environ:
    os.environ["CUSTOM_ASSET_FILES_DIR"]= "/home/jovyan/work/data"

if not "POLYGON_TICKERS_DIR" in os.environ:
    os.environ["POLYGON_TICKERS_DIR"]= "/home/jovyan/work/data/tickers"

config = PolygonConfig(
    environ=os.environ,
    calendar_name="NYSE",
    start_date="2018-01-01",
    end_date="2023-12-31",
    agg_time="1min",
)

pd.set_option('display.width', 250)

# print(f"{config.minute_aggs_dir=}")
# print(f"{config.trades_dir=}")
print(f"{config.custom_aggs_dir=}")
# print(f"{config.aggs_dir=}")
# print(f"{config.by_ticker_dir=}")
print(f"{config.tickers_csv_path=}")

config.custom_aggs_dir='/home/jovyan/work/data/60sec_aggs'
config.tickers_csv_path='/home/jovyan/work/data/tickers/tickers_2018-01-01_2023-12-31.csv'


In [5]:
tickers = get_ticker_universe(config, fetch_missing=True)
tickers

tickers_csv_path='/home/jovyan/work/data/tickers/tickers_2018-01-01_2023-12-31.csv'
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 49789 entries, ('CpP', 'XNYS', '0000831001', 'PFD', nan, True) to ('ZZrw', 'XNYS', '0000748015', 'RIGHT', nan, False)
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   start_date        49789 non-null  datetime64[ns]
 1   end_date          49789 non-null  datetime64[ns]
 2   last_updated_utc  49789 non-null  object        
 3   name              49789 non-null  object        
 4   share_class_figi  49789 non-null  object        
 5   delisted_utc      49789 non-null  object        
 6   currency_name     49789 non-null  object        
 7   locale            49789 non-null  object        
 8   market            49789 non-null  object        
dtypes: datetime64[ns](2), object(7)
memory usage: 6.3+ MB


,,,,,,start_date,end_date,last_updated_utc,name,share_class_figi,delisted_utc,currency_name,locale,market
ticker,primary_exchange,cik,type,composite_figi,active,,,,,,,,,
CpP,XNYS,0000831001,PFD,NaN,True,2018-01-02,2018-01-02,2024-12-03,[Citigroup Inc],[],[],[usd],[us],[stocks]
XKII,ARCX,0001081019,NaN,BBG00JHNY9Z4,True,2018-01-02,2018-01-02,2024-12-03,[SPDR Kensho Intelligent Structures ETF],[BBG00JHNYBP0],[],[usd],[us],[stocks]
XKST,ARCX,0001081019,NaN,BBG00JHP5PG6,True,2018-01-02,2018-01-02,2024-12-03,[SPDR Kensho Smart Mobility ETF],[BBG00JHP5Q56],[],[usd],[us],[stocks]
LDRS,ARCX,0001415726,NaN,UNKNOWN,True,2018-01-02,2018-01-02,2024-12-03,[Innovator IBD� ETF Leaders ETF],[UNKNOWN],[],[usd],[us],[stocks]
XKFS,ARCX,0001081019,NaN,BBG00JHP6203,True,2018-01-02,2018-01-02,2024-12-03,[SPDR Kensho Future Security ETF],[BBG00JHP62Q5],[],[usd],[us],[stocks]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GOEV,XNAS,0001750153,CS,BBG00NBK0GC3,True,2020-12-22,2023-12-29,2024-12-03,[Canoo Inc. Class A Common Stock],[BBG00NBK0H68],[],[usd],[us],[stocks]
GOEVW,XNAS,0001750153,WARRANT,BBG00NWRF3R2,True,2020-12-22,2023-12-29,2024-12-03,[Canoo Inc. Warrant],[],[],[usd],[us],[stocks]
GOEX,ARCX,0001432353,ETF,BBG0018HBWX2,True,2018-01-02,2023-12-29,2024-12-03,[Global X Gold Explorers ETF (NEW)],[BBG001TF8DG8],[],[usd],[us],[stocks]


In [6]:
tickers = get_ticker_universe(config, fetch_missing=True)
tickers

tickers_csv_path='/home/jovyan/work/data/tickers/tickers_2018-01-01_2023-12-31.csv'
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 49789 entries, ('CpP', 'XNYS', '0000831001', 'PFD', nan, True) to ('ZZrw', 'XNYS', '0000748015', 'RIGHT', nan, False)
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   start_date        49789 non-null  datetime64[ns]
 1   end_date          49789 non-null  datetime64[ns]
 2   last_updated_utc  49789 non-null  object        
 3   name              49789 non-null  object        
 4   share_class_figi  49789 non-null  object        
 5   delisted_utc      49789 non-null  object        
 6   currency_name     49789 non-null  object        
 7   locale            49789 non-null  object        
 8   market            49789 non-null  object        
dtypes: datetime64[ns](2), object(7)
memory usage: 6.3+ MB


,,,,,,start_date,end_date,last_updated_utc,name,share_class_figi,delisted_utc,currency_name,locale,market
ticker,primary_exchange,cik,type,composite_figi,active,,,,,,,,,
CpP,XNYS,0000831001,PFD,NaN,True,2018-01-02,2018-01-02,2024-12-03,[Citigroup Inc],[],[],[usd],[us],[stocks]
XKII,ARCX,0001081019,NaN,BBG00JHNY9Z4,True,2018-01-02,2018-01-02,2024-12-03,[SPDR Kensho Intelligent Structures ETF],[BBG00JHNYBP0],[],[usd],[us],[stocks]
XKST,ARCX,0001081019,NaN,BBG00JHP5PG6,True,2018-01-02,2018-01-02,2024-12-03,[SPDR Kensho Smart Mobility ETF],[BBG00JHP5Q56],[],[usd],[us],[stocks]
LDRS,ARCX,0001415726,NaN,UNKNOWN,True,2018-01-02,2018-01-02,2024-12-03,[Innovator IBD� ETF Leaders ETF],[UNKNOWN],[],[usd],[us],[stocks]
XKFS,ARCX,0001081019,NaN,BBG00JHP6203,True,2018-01-02,2018-01-02,2024-12-03,[SPDR Kensho Future Security ETF],[BBG00JHP62Q5],[],[usd],[us],[stocks]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GOEV,XNAS,0001750153,CS,BBG00NBK0GC3,True,2020-12-22,2023-12-29,2024-12-03,[Canoo Inc. Class A Common Stock],[BBG00NBK0H68],[],[usd],[us],[stocks]
GOEVW,XNAS,0001750153,WARRANT,BBG00NWRF3R2,True,2020-12-22,2023-12-29,2024-12-03,[Canoo Inc. Warrant],[],[],[usd],[us],[stocks]
GOEX,ARCX,0001432353,ETF,BBG0018HBWX2,True,2018-01-02,2023-12-29,2024-12-03,[Global X Gold Explorers ETF (NEW)],[BBG001TF8DG8],[],[usd],[us],[stocks]


# Test Tickers
Need to use is_test from the Ticker Details JSON response because it isn't in their model class.

https://www.nasdaqtrader.com/Trader.aspx?id=SymbolDirDefs

https://github.com/polygon-io/client-python/blob/master/polygon/rest/models/tickers.py#L87

Also Ticker Details works for the base symbol but not always for the extensions:
This works:
https://api.polygon.io/v3/reference/tickers/ATEST.A?apiKey=$POLYGON_API_KEY
But not this:
https://api.polygon.io/v3/reference/tickers/NTEST.B?date=2022-10-19&apiKey=$POLYGON_API_KEY

Does this deal with test tickers?
https://alpaca.markets/sdks/python/api_reference/trading/models.html#alpaca.trading.models.Asset.tradable

In [7]:
[ticker for ticker in tickers.index.get_level_values('ticker').to_list() if "TEST" in ticker]

['NTEST.C',
 'NTEST.B',
 'NTEST.A',
 'NTEST',
 'ATEST',
 'ATEST.A',
 'ATEST.B',
 'ATEST.C']

In [8]:
tickers[tickers.index.get_level_values('ticker').str.contains("TEST")].sort_index()

,,,,,,start_date,end_date,last_updated_utc,name,share_class_figi,delisted_utc,currency_name,locale,market
ticker,primary_exchange,cik,type,composite_figi,active,,,,,,,,,
ATEST,XASE,0001415311,NaN,BBG000Q2HK03,False,2018-01-02,2023-12-29,2016-10-25,[None],[BBG001S6X7W6],[2016-10-25],[usd],[us],[stocks]
ATEST.A,XASE,NaN,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]
ATEST.B,XASE,0000849998,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]
ATEST.C,XASE,0000860188,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]
NTEST,XNYS,0001515980,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]
NTEST.A,XNYS,0001566897,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]
NTEST.B,XNYS,0000807884,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]
NTEST.C,XNYS,0000805264,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]


In [9]:
tickers[tickers.index.get_level_values('ticker').str.contains("ZZT")].sort_index()

,,,,,,start_date,end_date,last_updated_utc,name,share_class_figi,delisted_utc,currency_name,locale,market
ticker,primary_exchange,cik,type,composite_figi,active,,,,,,,,,


In [10]:
table = pa_parquet.read_table("/home/jovyan/work/data/60sec_aggs/year=2024/month=12/date=2024-12-23/part-0.parquet")
table

pyarrow.Table
ticker: string not null
window_start: timestamp[ns, tz=UTC]
open: double
high: double
low: double
close: double
total: double
volume: int64
transactions: int64
vwap: double
year: dictionary<values=int32, indices=int32, ordered=0>
month: dictionary<values=int32, indices=int32, ordered=0>
date: dictionary<values=string, indices=int32, ordered=0>
----
ticker: [["RUM","RUN","RZLV","SAIC","SBUX",...,"ZSL","ZTAX","ZTEST","ZTST","ZVOL"],["XLI","XLV","XMTR","XOM","XP",...,"TWM","TXN","U","UAVS","UBER"],...,["ACIU","ACIW","ACLS","ACLX","ACMR",...,"UNIT","UNL","UNM","UNP","UP"],["RBB","RBBN","RBC","RBLX","RBRK",...,"FFSM","FFTY","FFWM","FG","FGEN"]]
window_start: [[2024-12-24 00:49:00.000000000Z,2024-12-24 00:49:00.000000000Z,2024-12-24 00:49:00.000000000Z,2024-12-24 00:49:00.000000000Z,2024-12-24 00:49:00.000000000Z,...,2024-12-24 01:00:00.000000000Z,2024-12-24 01:00:00.000000000Z,2024-12-24 01:00:00.000000000Z,2024-12-24 01:00:00.000000000Z,2024-12-24 01:00:00.000000000Z],[2024-1

In [11]:
table.to_pandas()

,ticker,window_start,open,high,low,close,total,volume,transactions,vwap,year,month,date
0,RUM,2024-12-24 00:49:00+00:00,13.070,13.070,13.070,13.070,5881.50,450,2,13.070000,2024,12,2024-12-23
1,RUN,2024-12-24 00:49:00+00:00,10.030,10.030,10.030,10.030,5015.00,500,1,10.030000,2024,12,2024-12-23
2,RZLV,2024-12-24 00:49:00+00:00,3.270,3.270,3.270,3.270,7916.67,2421,2,3.270000,2024,12,2024-12-23
3,SAIC,2024-12-24 00:49:00+00:00,111.700,111.700,111.700,111.700,335.10,3,1,111.700000,2024,12,2024-12-23
4,SBUX,2024-12-24 00:49:00+00:00,87.490,87.490,87.490,87.490,1399.84,16,1,87.490000,2024,12,2024-12-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972232,FFSM,2024-12-23 19:58:00+00:00,26.920,26.920,26.920,26.920,134.60,5,2,26.920000,2024,12,2024-12-23
1972233,FFTY,2024-12-23 19:58:00+00:00,29.820,29.820,29.820,29.820,2982.00,100,2,29.820000,2024,12,2024-12-23
1972234,FFWM,2024-12-23 19:58:00+00:00,5.980,5.980,5.970,5.970,1289.68,216,3,5.970741,2024,12,2024-12-23
1972235,FG,2024-12-23 19:58:00+00:00,41.540,41.560,41.540,41.550,290.85,7,3,41.550000,2024,12,2024-12-23


In [12]:
table = pa_parquet.read_table("/home/jovyan/work/data/60sec_aggs_signals/year=2022/month=9/date=2022-09-02/part-0.parquet")
table

pyarrow.Table
open: double
high: double
low: double
close: double
total: double
volume: double
transactions: double
vwap: double
TP: double
ret1bar: double
ret2bar: double
ret3bar: double
ret4bar: double
ret5bar: double
ret6bar: double
ret7bar: double
ret8bar: double
ret9bar: double
ret10bar: double
ret11bar: double
ret12bar: double
ret13bar: double
NATR: double
taCCI: double
CCI: double
MFI: double
STOCHk: double
STOCHd: double
STOCHh: double
ticker: string
window_start: timestamp[ns, tz=UTC]
year: dictionary<values=int32, indices=int32, ordered=0>
month: dictionary<values=int32, indices=int32, ordered=0>
date: dictionary<values=string, indices=int32, ordered=0>
----
open: [[73.64,73.64,73.64,73.64,73.64,...,9.09,9.09,9.09,9.09,9.08],[128.93,48.72,48.72,48.72,48.72,...,38.87,38.87,38.87,38.87,38.87],...,[100.03,100.02,100.02,100.02,100.02,...,0.1545,0.1545,0.1545,0.1545,0.1545],[38.72,38.72,38.72,38.72,38.72,...,73.66,73.66,73.66,73.64,73.64]]
high: [[73.64,73.64,73.64,73.64,73.64,...

In [14]:
df = table.to_pandas()
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536240 entries, 0 to 536239
Data columns (total 34 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   open          536240 non-null  float64            
 1   high          536240 non-null  float64            
 2   low           536240 non-null  float64            
 3   close         536240 non-null  float64            
 4   total         536240 non-null  float64            
 5   volume        536240 non-null  float64            
 6   transactions  536240 non-null  float64            
 7   vwap          536240 non-null  float64            
 8   TP            536240 non-null  float64            
 9   ret1bar       536240 non-null  float64            
 10  ret2bar       531920 non-null  float64            
 11  ret3bar       528955 non-null  float64            
 12  ret4bar       526037 non-null  float64            
 13  ret5bar       523146 non-null  float64      

,open,high,low,close,total,volume,transactions,vwap,TP,ret1bar,...,CCI,MFI,STOCHk,STOCHd,STOCHh,ticker,window_start,year,month,date
0,73.64,73.64,73.64,73.64,0.0,0.0,0.0,73.640000,73.640000,0.000000,...,28.025191,9.092368,66.666667,74.074074,-7.407407,XLP,2022-09-02 11:55:00+00:00,2022,9,2022-09-02
1,73.64,73.64,73.64,73.64,0.0,0.0,0.0,73.640000,73.640000,0.000000,...,22.386224,9.092368,66.666667,68.518519,-1.851852,XLP,2022-09-02 11:56:00+00:00,2022,9,2022-09-02
2,73.64,73.64,73.64,73.64,0.0,0.0,0.0,73.640000,73.640000,0.000000,...,16.214689,9.092368,66.666667,66.666667,0.000000,XLP,2022-09-02 11:57:00+00:00,2022,9,2022-09-02
3,73.64,73.64,73.64,73.64,0.0,0.0,0.0,73.640000,73.640000,0.000000,...,8.289794,9.092368,66.666667,66.666667,0.000000,XLP,2022-09-02 11:58:00+00:00,2022,9,2022-09-02
4,73.64,73.64,73.64,73.64,0.0,0.0,0.0,73.640000,73.640000,0.000000,...,-4.444444,9.092368,66.666667,66.666667,0.000000,XLP,2022-09-02 11:59:00+00:00,2022,9,2022-09-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536235,73.66,73.66,73.66,73.66,0.0,0.0,0.0,73.660000,73.660000,0.000000,...,155.555556,100.000000,100.000000,66.666667,33.333333,XLP,2022-09-02 11:50:00+00:00,2022,9,2022-09-02
536236,73.66,73.66,73.66,73.66,0.0,0.0,0.0,73.660000,73.660000,0.000000,...,116.666667,100.000000,100.000000,88.888889,11.111111,XLP,2022-09-02 11:51:00+00:00,2022,9,2022-09-02
536237,73.66,73.66,73.65,73.65,44191.0,600.0,3.0,73.651667,73.653333,-0.000136,...,76.740741,14.287100,94.444444,98.148148,-3.703704,XLP,2022-09-02 11:52:00+00:00,2022,9,2022-09-02
536238,73.64,73.64,73.64,73.64,29456.0,400.0,4.0,73.640000,73.640000,0.000000,...,40.159129,9.092368,83.333333,92.592593,-9.259259,XLP,2022-09-02 11:53:00+00:00,2022,9,2022-09-02


In [15]:
table.schema

open: double
high: double
low: double
close: double
total: double
volume: double
transactions: double
vwap: double
TP: double
ret1bar: double
ret2bar: double
ret3bar: double
ret4bar: double
ret5bar: double
ret6bar: double
ret7bar: double
ret8bar: double
ret9bar: double
ret10bar: double
ret11bar: double
ret12bar: double
ret13bar: double
NATR: double
taCCI: double
CCI: double
MFI: double
STOCHk: double
STOCHd: double
STOCHh: double
ticker: string
window_start: timestamp[ns, tz=UTC]
year: dictionary<values=int32, indices=int32, ordered=0>
month: dictionary<values=int32, indices=int32, ordered=0>
date: dictionary<values=string, indices=int32, ordered=0>
-- schema metadata --
date: '2022-09-02'

In [22]:
table = pa_parquet.read_table("/home/jovyan/work/data/60sec_aggs_labels/labels_2024-12-01_2024-12-07.parquet")
table

pyarrow.Table
gain: double
duration: int64
volume: double
traded_value: double
end_dt: timestamp[ns, tz=UTC]
previous_price: double
start_price: double
end_price: double
next_price: double
prices: list<element: double>
  child 0, element: double
ticker: string
start_dt: timestamp[ns, tz=UTC]
----
gain: [[18.40834217074176,49.9278047772196,27.089857149813685,17.64752790839771,17.4710586254166,...,29.603611215856276,23.264311792672235,43.506656875428284,18.89529941475377,36.80808946098768]]
duration: [[9,9,9,3,45,...,9,7,108,4,5]]
volume: [[840916,105797,931024,248802,52180,...,3108754,1288873,731656,2236775,1344327]]
traded_value: [[962518.3835000002,661502.9399999992,5862515.86,225466.1170000003,161212.2,...,12340469.340000011,4356865.320000003,1447175.0859999985,4906662.222000001,7837090.922999993]]
end_dt: [[2024-12-02 11:47:00.000000000Z,2024-12-02 10:10:00.000000000Z,2024-12-02 10:32:00.000000000Z,2024-12-02 09:08:00.000000000Z,2024-12-02 12:46:00.000000000Z,...,2024-12-06 10:02:00

In [23]:
df = table.to_pandas()
df.info()
df

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 120 entries, ('BREA', Timestamp('2024-12-02 11:38:00+0000', tz='UTC')) to ('ZBAO', Timestamp('2024-12-06 11:59:00+0000', tz='UTC'))
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gain            120 non-null    float64            
 1   duration        120 non-null    int64              
 2   volume          120 non-null    float64            
 3   traded_value    120 non-null    float64            
 4   end_dt          120 non-null    datetime64[ns, UTC]
 5   previous_price  120 non-null    float64            
 6   start_price     120 non-null    float64            
 7   end_price       120 non-null    float64            
 8   next_price      120 non-null    float64            
 9   prices          120 non-null    object             
dtypes: datetime64[ns, UTC](1), float64(7), int64(1), object(1)
memory usage: 17.2+ KB


gain  duration     volume  traded_value                    end_dt  previous_price  start_price  end_price  next_price                                             prices
ticker start_dt                                                                                                                                                                                                
BREA   2024-12-02 11:38:00+00:00  18.408342         9   840916.0  9.625184e+05 2024-12-02 11:47:00+00:00        0.999967     0.999936   1.184008    1.125179  [1.0, 1.0, 1.001, 1.019, 1.029, 1.035, 1.103, ...
EHGO   2024-12-02 10:01:00+00:00  49.927805         9   105797.0  6.615029e+05 2024-12-02 10:10:00+00:00        4.250000     4.249091   6.370569    5.525681  [4.249, 4.251, nan, nan, nan, nan, 4.493, 4.90...
       2024-12-02 10:23:00+00:00  27.089857         9   931024.0  5.862516e+06 2024-12-02 10:32:00+00:00        5.345565     5.345066   6.793037    6.682072  [5.345, 5.565, 5.84, 5.983, 6.227, 6.431, 6.44...
HOVR   2024-12-02 09:05:00+00:00  17.647528         3   248802.0  2.254661e+05 2024-12-02 09:08:00+00:00        0.782486     0.780194   0.917879    0.903561                        [0.78, 0.783, 0.802, 0.918]
JFBR   2024-12-02 12:01:00+00:00  17.471059        45    52180.0  1.612122e+05 2024-12-02 12:46:00+00:00        2.674118     2.660000   3.124730    2.916975  [2.66, nan, nan, nan, nan, nan, nan, nan, nan,...
...                                     ...       ...        ...           ...                       ...             ...          ...        ...         ...                                                ...
UPC    2024-12-06 09:53:00+00:00  29.603611         9  3108754.0  1.234047e+07 2024-12-06 10:02:00+00:00        3.349723     3.341295   4.330439    4.145115  [3.341, 3.539, 3.777, 3.839, 3.896, 3.913, 4.0...
       2024-12-06 13:06:00+00:00  23.264312         7  1288873.0  4.356865e+06 2024-12-06 13:13:00+00:00        3.051410     2.922688   3.602631    3.541086  [2.923, 3.014, 3.065, 3.202, 3.328, 3.365, 3.5...
VSEE   2024-12-06 11:15:00+00:00  43.506657       108   731656.0  1.447175e+06 2024-12-06 13:03:00+00:00        1.476136     1.455537   2.088793    1.954362  [1.456, 1.45, nan, nan, nan, nan, nan, nan, na...
       2024-12-06 13:04:00+00:00  18.895299         4  2236775.0  4.906662e+06 2024-12-06 13:08:00+00:00        2.088793     1.954362   2.323644    2.310468                [1.954, 2.128, 2.177, 2.295, 2.324]
ZBAO   2024-12-06 11:59:00+00:00  36.808089         5  1344327.0  7.837091e+06 2024-12-06 12:04:00+00:00        4.724978     4.522039   6.186515    5.658022          [4.522, 4.891, 5.359, 5.92, 6.134, 6.187]

[120 rows x 10 columns]

In [24]:
show(df)